<a href="https://colab.research.google.com/github/RobertoAlessandri/CNN_DOA/blob/main/FeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import numpy as np
import matplotlib.pyplot as plt
import librosa, librosa.display
import IPython.display as ipd
from pathlib import Path
import sklearn
plt.style.use('seaborn')

In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
%cd /content/drive/MyDrive/Github/CNN_DOA/Dataset/TRAIN


# Get a list of all audio files and get the class label for each file
audiofiles = [str(file) for file in Path().glob('SA*.wav')]
labels = []
for i,file in enumerate(audiofiles):
  fileid = file.split('.wav')[-2]
  target = fileid.split('-')[-1]
  labels.append(int(target))

/content/drive/MyDrive/Github/CNN_DOA/Dataset/TRAIN


In [86]:
audiofiles

['SA1400-0-350-2-7.wav',
 'SA1401-1-350-2-7.wav',
 'SA1402-2-350-2-7.wav',
 'SA1403-3-350-2-7.wav',
 'SA1404-0-351-7-21.wav',
 'SA1405-1-351-7-21.wav',
 'SA1406-2-351-7-21.wav',
 'SA1407-3-351-7-21.wav',
 'SA1408-0-352-4-13.wav',
 'SA1409-1-352-4-13.wav',
 'SA1410-2-352-4-13.wav',
 'SA1411-3-352-4-13.wav',
 'SA1412-0-353-4-12.wav',
 'SA1413-1-353-4-12.wav',
 'SA1414-2-353-4-12.wav',
 'SA1415-3-353-4-12.wav',
 'SA1416-0-354-8-23.wav',
 'SA1417-1-354-8-23.wav',
 'SA1418-2-354-8-23.wav',
 'SA1419-3-354-8-23.wav',
 'SA1420-0-355-1-4.wav',
 'SA1421-1-355-1-4.wav',
 'SA1422-2-355-1-4.wav',
 'SA1423-3-355-1-4.wav',
 'SA1424-0-356-6-18.wav',
 'SA1425-1-356-6-18.wav',
 'SA1426-2-356-6-18.wav',
 'SA1427-3-356-6-18.wav',
 'SA1428-0-357-7-21.wav',
 'SA1429-1-357-7-21.wav',
 'SA1430-2-357-7-21.wav',
 'SA1431-3-357-7-21.wav',
 'SA1432-0-358-9-28.wav',
 'SA1433-1-358-9-28.wav',
 'SA1434-2-358-9-28.wav',
 'SA1435-3-358-9-28.wav',
 'SA1436-0-359-8-23.wav',
 'SA1437-1-359-8-23.wav',
 'SA1438-2-359-8-23.

In [87]:
print(len(audiofiles))

4800


In [88]:
class_34 = [str(file) for file in Path().glob('SA*-34.wav')]
class_15 = [str(file) for file in Path().glob('SA*-15.wav')]

print('Number of class_0 examples: {}'.format(len(class_34)))
print('Number of class_1 examples: {}'.format(len(class_15)))

print('class_34 files:')
print(*class_34, sep='\n')


print('class_7 files:')
print(*class_15, sep='\n')

Number of class_0 examples: 480
Number of class_1 examples: 240
class_34 files:
SA1448-0-362-11-34.wav
SA1449-1-362-11-34.wav
SA1450-2-362-11-34.wav
SA1451-3-362-11-34.wav
SA1460-0-365-11-34.wav
SA1461-1-365-11-34.wav
SA1462-2-365-11-34.wav
SA1463-3-365-11-34.wav
SA1528-0-382-11-34.wav
SA1529-1-382-11-34.wav
SA1530-2-382-11-34.wav
SA1531-3-382-11-34.wav
SA1540-0-385-11-34.wav
SA1541-1-385-11-34.wav
SA1542-2-385-11-34.wav
SA1543-3-385-11-34.wav
SA1608-0-402-11-34.wav
SA1609-1-402-11-34.wav
SA1610-2-402-11-34.wav
SA1611-3-402-11-34.wav
SA1620-0-405-11-34.wav
SA1621-1-405-11-34.wav
SA1622-2-405-11-34.wav
SA1623-3-405-11-34.wav
SA1688-0-422-11-34.wav
SA1689-1-422-11-34.wav
SA1690-2-422-11-34.wav
SA1691-3-422-11-34.wav
SA1700-0-425-11-34.wav
SA1701-1-425-11-34.wav
SA1702-2-425-11-34.wav
SA1703-3-425-11-34.wav
SA1768-0-442-11-34.wav
SA1769-1-442-11-34.wav
SA1770-2-442-11-34.wav
SA1771-3-442-11-34.wav
SA1780-0-445-11-34.wav
SA1781-1-445-11-34.wav
SA1782-2-445-11-34.wav
SA1783-3-445-11-34.wav


## Flow:
1) microphone signals

2) STFT 

3) Active SI estimation

4) Whitening weighting

In [89]:
print(len(class_15))
print(type(class_15))
print(len(class_15[0]))
print(type(class_15[0]))

240
<class 'list'>
21
<class 'str'>


In [90]:
sr = 16000

def stringToSignal(listclass):
  mic_0 = []
  mic_1 = []
  mic_2 = []
  mic_3 = []
  for i in range(len(listclass)):
    if listclass[i].find("-0-", 3, 10) != -1 :
      mic_0.append(listclass[i])
    if listclass[i].find("-1-", 3, 10) != -1 :
      mic_1.append(listclass[i])
    if listclass[i].find("-2-", 3, 10) != -1 :
      mic_2.append(listclass[i])
    if listclass[i].find("-3-", 3, 10) != -1 :
      mic_3.append(listclass[i])  
  p_0 = list(librosa.load(p, None)[0] for p in mic_0)
  p_1 = list(librosa.load(p, None)[0] for p in mic_1)
  p_2 = list(librosa.load(p, None)[0] for p in mic_2)
  p_3 = list(librosa.load(p, None)[0] for p in mic_3)
  return p_0, p_1, p_2, p_3

In [91]:
def featureExtractor(p_0, p_1, p_2, p_3):
  # STFT 3-1 for x axis, 4 -2 for y axis <- velocity
  d = 0.04
  ro = 1.225
  Vx = []
  Vy = []
  signals_len = len(p_0)
  for i in range(signals_len):
    Vx.append((librosa.stft(p_2[i]) - librosa.stft(p_0[i])).imag / (d*ro))
    Vy.append((librosa.stft(p_3[i]) - librosa.stft(p_1[i])).imag / (d*ro))
  
  # I can simply do the average of the STFT
  P_0 = []
  P_1 = []
  P_2 = []
  P_3 = []
  P0 = []
  for i in range(signals_len):
    P_0.append(librosa.stft(p_0[i]))
    P_1.append(librosa.stft(p_1[i]))
    P_2.append(librosa.stft(p_2[i]))
    P_3.append(librosa.stft(p_3[i]))
    P0.append((P_0[i] + P_1[i] + P_2[i] + P_3[i])/4)

  # I0x(omega,t) = P0(omega,t)*conj(Vx(omgea,t))
  I0x = []
  I0y = []
  for i in range(signals_len):
    I0x.append(P0[i]*Vx[i].conjugate())
    I0y.append(P0[i]*Vy[i].conjugate())

  # According to the theory on Sound Intensity (SI), only the real part (active 
  # intensity vector) contains the location information of a sound source.

  # Ix(omega,t) = Re{I0x(omgea,t)}
  Ix = []
  Iy = []
  for i in range(signals_len):
    Ix.append(I0x[i].real)
    Iy.append(I0y[i].real)

  # Whitening weighting 
  beta = 1e6
  W = []
  for i in range(signals_len):
    W.append((np.abs(P0[i])**2 + beta * (np.abs(Vx[i])**2 + np.abs(Vy[i])**2))**(1/2))

  # IxWeighted(omega,t) = Ix(omega,t)/weight
  IxW = []
  IyW = []
  for i in range(signals_len):
    IxW.append(Ix[i]/W[i])
    IyW.append(Iy[i]/W[i])

  return IxW, IyW



In [92]:
p_0, p_1, p_2, p_3 = stringToSignal(class_15)
IxW, IyW = featureExtractor(p_0, p_1, p_2, p_3)


In [93]:
(np.asarray(IxW).shape)

(60, 1025, 16)

In [94]:
ipd.display(ipd.Audio(p_3[50], rate=sr))
print(len(p_0))
print(len(p_0[10]))

60
8000


In [95]:

# Averaging Micropone signals to get the pressure at the origin
signals_len = len(p_0)
p0 = []
for i in range(signals_len):
  p0.append((p_0[i] + p_1[i] + p_2[i] + p_3[i])/4)

In [96]:
ipd.display(ipd.Audio(p0[50], rate=sr))
print(len(p0))
print(len(p0[10]))

60
8000


In [97]:
print(signals_len)
print(np.asarray(p_0).shape)


60
(60, 8000)


According to the theory on Sound Intensity (SI), only the real part (active intensity vector) contains the location information of a sound source.

In [98]:
print(np.asarray(IxW).shape)
print(np.asarray(IxW[0]).shape)

(60, 1025, 16)
(1025, 16)


## Subarrays


Actually, only three microphones are enough to estimate the SI by forming two rothogonal first-order DMAs with a shared microphone.

A = 2,1,4
B = 3,2,1
C = 4,3,2
D = 1,4,3

A) case (1 shared)

2,1 form  the first-order DMA along the direction 3π/4 and 4,1 along the direction -3π/4.

In [99]:
d = 0.04
ro = 1.225
Vx = [] # -3π/4,M1
Vy = [] # 3π/4,M1
sqrt2 = np.sqrt(2)
for i in range(signals_len):
  Vx.append(sqrt2 * (librosa.stft(p_2[i]) - librosa.stft(p_0[i])).imag / (d*ro))
  Vy.append(sqrt2 * (librosa.stft(p_3[i]) - librosa.stft(p_1[i])).imag / (d*ro))

sqrt2 = np.sqrt(2)



In [100]:
def featureExtractorSubArrays(a, b, c):
  # STFT 3-1 for x axis, 4 -2 for y axis <- velocity
  d = 0.04
  ro = 1.225
  Vx = []
  Vy = []
  signals_len = len(p_0)
  for i in range(signals_len):
    Vx.append((librosa.stft(c[i]) - librosa.stft(b[i])).imag / (d*ro))
    Vy.append((librosa.stft(b[i]) - librosa.stft(a[i])).imag / (d*ro))
  
  # I can simply do the average of the STFT
  A = []
  B = []
  C = []
  P0 = []
  for i in range(signals_len):
    A.append(librosa.stft(a[i]))
    B.append(librosa.stft(b[i]))
    C.append(librosa.stft(c[i]))
    P0.append((A[i] + B[i] + C[i])/3)

  # I0x(omega,t) = P0(omega,t)*conj(Vx(omgea,t))
  I0x = []
  I0y = []
  for i in range(signals_len):
    I0x.append(P0[i]*Vx[i].conjugate())
    I0y.append(P0[i]*Vy[i].conjugate())

  # According to the theory on Sound Intensity (SI), only the real part (active 
  # intensity vector) contains the location information of a sound source.

  # Ix(omega,t) = Re{I0x(omgea,t)}
  Ix = []
  Iy = []
  for i in range(signals_len):
    Ix.append(I0x[i].real)
    Iy.append(I0y[i].real)

  # Whitening weighting 
  beta = 1e6
  W = []
  for i in range(signals_len):
    W.append((np.abs(P0[i])**2 + beta * (np.abs(Vx[i])**2 + np.abs(Vy[i])**2))**(1/2))

  # IxWeighted(omega,t) = Ix(omega,t)/weight
  IxW = []
  IyW = []
  for i in range(signals_len):
    IxW.append(Ix[i]/W[i])
    IyW.append(Iy[i]/W[i])

  return IxW, IyW

In [101]:
IxW_minus3πfourth_M0, IxW_3πfourth_M0 = featureExtractorSubArrays(p_1, p_0, p_3)

B) case (2 shared)

1,2 form  the first-order DMA along the direction -π/4 and 3,2 along the direction -3π/4.

In [102]:
IxW_minusπfourth_M1, IxW_minus3πfourth_M1 = featureExtractorSubArrays(p_2, p_1, p_0)

C) case (3 shared)

2,3 form  the first-order DMA along the direction π/4 and 4,3 along the direction -π/4.

In [103]:
IxW_πfourth_M2, IxW_minusπfourth_M2 = featureExtractorSubArrays(p_3, p_2, p_1)

D) case (4 shared)

1,4 form  the first-order DMA along the direction π/4 and 3,4 along the direction 3π/4.

In [104]:
IxW_πfourth_M3, IxW_3πfourth_M3 = featureExtractorSubArrays(p_0, p_3, p_2)

In [105]:
np.shape(IxW[i].T)

(16, 1025)

In [106]:
len(IxW[0])

1025

In [107]:
I_removed = IxW[0][1:1025]
print(len(I_removed))

1024


## Summary of the features

Since the SI features at zero frequency are equal to 0 and hence contain no source location information, the SI features at zero frequency are discarded.

Moreover, considering the conjugate symmetrix property of the Fourier Transform, it suffices to utilize only the SI features over the frequency bins of positive frequencies.

In [110]:
def GammaCreator(Gamma, I1, I2, I3, I4, I5, I6, I7, I8, I9, I10):

  Gamma[:, :, 0] = I1[1:1025].T
  Gamma[:, :, 1] = I2[1:1025].T
  Gamma[:, :, 2] = I3[1:1025].T
  Gamma[:, :, 3] = I4[1:1025].T
  Gamma[:, :, 4] = I5[1:1025].T
  Gamma[:, :, 5] = I6[1:1025].T
  Gamma[:, :, 6] = I7[1:1025].T
  Gamma[:, :, 7] = I8[1:1025].T
  Gamma[:, :, 8] = I9[1:1025].T
  Gamma[:, :, 9] = I10[1:1025].T

  return Gamma


In [112]:
N = 16
M = 2048
C = 10
Gamma = np.empty([N, int(M/2 + 1) - 1, C], dtype = 'complex_')
Gamma_container = np.empty([signals_len, N, int(M/2 + 1) - 1, C], dtype = 'complex_')

for i in range(signals_len):
  Gamma_container[i] = GammaCreator(Gamma, IxW[i], IyW[i], IxW_minus3πfourth_M0[i], 
                                    IxW_3πfourth_M0[i], IxW_minusπfourth_M1[i], 
                                    IxW_minus3πfourth_M1[i], IxW_πfourth_M2[i], 
                                    IxW_minusπfourth_M2[i], IxW_πfourth_M3[i], 
                                    IxW_3πfourth_M3[i])

In [113]:
np.shape(Gamma_container)

(60, 16, 1024, 10)